In [ ]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.cm as cm
import time
import re

completed=[]


style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from Config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
completed

In [ ]:
#this function defines the account holder
mention=api.mentions_timeline(count=25)
accountHolder=mention[0]['user']['screen_name']
accountHolder

In [ ]:
#This function retrieves the data using the API

users=[]

mention=api.mentions_timeline(count=25)

#this function will search for any words in recent mentions that follows the symbol " @ "
#removing references to the account holder

for tweet in mention:
    wordList= (tweet["text"])
    source= (tweet['user']['screen_name'])
    x=re.findall(r"@(\w+)", wordList)
    for y in x:
        if y not in users and y!=accountHolder:
            users.append({"user":y,"Tweeted By":source})
            print (f"@{source} mentioned @{y}")

In [ ]:
#print out of active requests

for user in users:
    u=(user['user'])
    _source=(user['Tweeted By'])
    if u not in completed :
        print(u)
        print(f"requested by {_source}")
    else:
        print ("Previously analysed")

In [ ]:
#this function add the results to the users list to be analayzed, 
#but excludes twitter accounts that has been previously anaylzed.

userList=[]
dateList=[]
compoundList=[]
postiveList=[]
negativeList=[]
neutralList=[]
newsOrgList=[]
tweetsAgoList=[]
sourceList=[]
target_user=[]


# Target Account
for user in users:
    u=(user['user'])
    _source=(user['Tweeted By'])
    if u not in completed:
        target_user.append(u)
        sourceList.append(_source)


for user in target_user:

        # Counter
        counter = 1


        # Variable for max_id
        oldest_tweet = None

        # Loop through 5 pages of tweets (total 100 tweets)
        for x in range(5):

            # Get all tweets from home feed
            try:
                public_tweets = api.user_timeline(user, max_id = oldest_tweet)
            except:
                continue

            # Loop through all tweets 
            for tweet in public_tweets:

                # Print Tweets
                # print("Tweet {counter}: {tweet["text"]})

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Get Tweet ID, subtract 1, and assign to oldest_tweet
                oldest_tweet = tweet['id'] - 1

                # Add sentiments for each tweet into a list
                userList.append(user)
                dateList.append(tweet["created_at"])
                compoundList.append(compound)
                postiveList.append(pos)
                negativeList.append(neg)
                neutralList.append(neu)
                tweetsAgoList.append(counter)
                completed.append(user)
                sourceList.append(_source)



                # Add to counter 
                counter += 1
        



In [ ]:
#list of completed twitter accounts
ListOfAnalyzedAccounts=set(completed)

ListOfAnalyzedAccounts

In [ ]:
#creating a dataframe of the Vader analysis

ZipList=list(zip(
userList,
dateList,
compoundList,
postiveList,
negativeList,
neutralList,
tweetsAgoList,
sourceList))

sentiments_pd= pd.DataFrame(ZipList, columns = ["User",
                                                "Created",
                                           "Compound",
                                               "Positive",
                                               "Negative",
                                               "Neutral",
                                               "Tweets ago",
                                               "Requested by"])


#displaying table
sentiments_pd.head()

In [ ]:
target_user

In [ ]:
for tweep in target_user:
    plotData=sentiments_pd.loc[sentiments_pd["User"]==tweep,:]
    requested=plotData["Requested by"].iloc[0]

  
    # Create plot
    x_vals = plotData["Tweets ago"]
    y_vals = plotData["Compound"]
    plt.plot(x_vals, y_vals, marker="o", linewidth=0.5, alpha=0.8)

    # # Incorporate the other graph properties
    now = datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M")
    plt.title(f"Sentiment Analysis of Tweets ({now}) for {tweep}")
    plt.xlim(0,105)
    plt.ylabel("Tweet Polarity")
    plt.xlabel("Tweets Ago")
    plt.savefig(f"{tweep} SentimentatAnalysis.png", pad_inches=0.5, bbox_inches="tight")
    plt.show()
    file=f"{tweep} SentimentatAnalysis.png"
    api.update_with_media(file, (f"Data Analytics tweet sentimentat analysis(BOT) requested by @{requested}"))



